In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


In [3]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [4]:
def build_model(hp):  
    model = keras.Sequential()
    model.add(layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=16, max_value=64, step=16),
                             kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
                             activation='relu', input_shape=(28,28,1)))
    model.add(layers.Conv2D(filters=hp.Int('conv_2_filter', min_value=16, max_value=64, step=16),
                             kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
                             activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=32),
                           activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [5]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=25,
    executions_per_trial=1,
    directory='my_dir',
    project_name='helloworld')

tuner.search_space_summary()


Search space summary
Default search space size: 6
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [6]:
tuner.search(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

best_model = tuner.get_best_models()[0]
best_hyperparameters = tuner.get_best_hyperparameters()[0]

print(best_hyperparameters.values)
test_loss, test_acc = best_model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


Trial 1 Complete [00h 04m 13s]
val_accuracy: 0.11349999904632568

Best val_accuracy So Far: 0.11349999904632568
Total elapsed time: 00h 04m 13s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
48                |48                |conv_1_filter
5                 |3                 |conv_1_kernel
32                |16                |conv_2_filter
5                 |3                 |conv_2_kernel
128               |64                |dense_1_units
0.0001            |0.01              |learning_rate

Epoch 1/5
1477/1875 [======================>.......] - ETA: 19s - loss: 0.3626 - accuracy: 0.9326

KeyboardInterrupt: 